In [123]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize
import time ## REMOVE THIS LATER

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.

CHECK DELIST:
- Purpose: checks that there is actually valid market data for this stock
- Code: if 'Close' not in data.columns or data['Close'].isnull().all():

PORTFOLIO GENERATOR
- Purpose: calculate how many shares of each stock to buy, and how much to pay in fees
- Code:

portfolio_value = 1000000 * weights  # Portfolio value based on optimized weights

shares = portfolio_val / np.array([ticker_prices[ticker] for ticker in selected_tickers])  # Number of shares

fee = np.minimum(3.95, 0.001 * shares)

portfolio_val -= fee


In [154]:
#October 01, 2023 to September 30, 2024, 2023

START_DATE = '2023-10-01'
END_DATE = '2024-11-20'

 # date on which we will take stock Close prices
TARGET_DATE = '2024-11-13'

INVESTMENT = 1000000

In [141]:
# filter out invalid stocks
def validity(tickers):

    # list to keep all of t
    # he valid stocks
    final_list = []

    for ticker in tickers:
        #only append the stock to the final stock list if it is isn't delisted, has a US or CAD currency, and has the required monthly share volume
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_volume(ticker)):
                final_list.append(ticker)
    return final_list

# checks if the consumed Ticker is delisted
def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        time.sleep(0.3)
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock
            if 'Close' not in data.columns or data['Close'].isnull().all():
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

# checks if the consumed Ticker meets the requirement of an average monthly share volume of 100,000 shares
def check_volume(ticker):
    volume = yf.Ticker(ticker).history(start=START_DATE, end='2024-09-30')['Volume']
    time.sleep(0.3)
    avg_monthly_volume = volume.resample('ME').mean()
    return avg_monthly_volume.mean() >= 100000

# checks whether or not the consumed Ticker is listed in CAD or in USD
def check_currency(ticker):
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # if there's an error retrieving the data, we can assume it doesnt satisfy the required monthly volume

# read in the provided Ticker file
all_data = pd.read_csv('Tickers_Example.csv', header=None)

# create a new DataFrame only containing the valid Tickers in the consumed CSV
all_data = pd.DataFrame(validity(all_data[0]))
all_data.rename(columns = {0:'Ticker'}, inplace=True)



$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [142]:
# create a DataFrame to store 
ticker_returns = pd.DataFrame(columns=all_data['Ticker'])

# calculate and store all returns for all all Tickers
for ticker in all_data['Ticker']:
        data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close']
        time.sleep(0.3)
        ticker_returns[ticker] = data.pct_change().dropna()  # this code calculates daily returns
        
ticker_returns.index = pd.to_datetime(ticker_returns.index.strftime("%Y-%m-%d"))

ticker_returns

Ticker,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-03,-0.007770,-0.005531,0.000627,-0.006585,-0.023481,-0.036614,-0.029636,0.005803,-0.029588,-0.006645,...,-0.008191,-0.024235,-0.046045,0.006083,-0.009452,-0.016881,-0.009911,0.005626,-0.008576,-0.024891
2023-10-04,0.007309,0.001764,-0.001461,0.010319,0.012278,0.018281,0.010893,-0.011592,0.001158,-0.020067,...,-0.003086,0.009280,0.025412,0.032093,-0.002636,0.004579,0.002257,-0.007410,-0.000195,-0.003191
2023-10-05,0.007198,-0.001625,0.005750,0.000873,0.013309,-0.008189,0.007259,-0.002356,0.004626,-0.112628,...,-0.000455,0.007707,-0.009968,0.019378,0.011595,-0.018105,0.010987,-0.008059,-0.002212,0.020166
2023-10-06,0.014751,0.005358,0.007069,0.008170,0.011970,0.015878,0.015161,0.005851,0.000384,-0.009615,...,0.009836,-0.001477,0.024612,-0.003095,0.005164,0.012185,0.016620,0.009071,0.005803,-0.000314
2023-10-09,0.008451,0.005869,-0.001238,-0.000577,0.003614,0.002344,-0.003348,0.005924,0.009206,NaN,...,0.008297,NaN,NaN,NaN,NaN,-0.001783,0.003239,0.009088,0.000454,0.004708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13,0.003969,-0.004325,-0.006012,0.026649,0.016431,0.024843,-0.003015,-0.035820,0.000218,0.014793,...,-0.018298,-0.001852,0.058715,-0.003199,0.002936,-0.033180,-0.014317,0.006279,0.027201,0.008077
2024-11-14,0.013770,-0.004227,-0.006134,-0.022014,-0.004468,-0.012237,0.002294,-0.013074,0.000654,-0.011662,...,0.022129,-0.002204,-0.048201,-0.021550,0.013747,0.001361,-0.020995,-0.020091,-0.007780,-0.016807
2024-11-15,-0.014109,-0.027354,0.007563,-0.023476,0.000264,-0.041942,-0.004960,0.014840,0.018518,-0.017699,...,-0.021162,-0.005347,-0.003650,0.002343,-0.010547,-0.023689,-0.001551,-0.000042,0.010781,-0.008149


In [143]:
def std_rank(tickers):
    std_devs = ticker_returns.std()
    
    # Create a DataFrame with tickers and their respective standard deviations
    ranked_stocks = pd.DataFrame({
        "ticker": std_devs.index,
        "STD": std_devs.values
    })
    
    # Sort by standard deviation in ascending order (lowest STD at the top)
    ranked_stocks = ranked_stocks.sort_values(by="STD", ascending=True)
    
    return ranked_stocks['ticker'].values[:24].tolist()


top_24 = std_rank(ticker_returns)

In [ ]:
# fetch TSX60 index information
tsx60_info = yf.Ticker('XIU.TO')
tsx60 = tsx60_info.history(start=START_DATE, end=END_DATE, interval='1d')
time.sleep(0.3)

# fetch SP500 index information
sp500_info = yf.Ticker('^GSPC')
sp500 = sp500_info.history(start=START_DATE, end=END_DATE, interval='1d')

# reindex sp500 so that it shares the same index as tsx60
sp500 = sp500.reindex(tsx60.index, method='nearest')

# calculate the average market return
market_returns = (tsx60['Close'].pct_change() + sp500['Close'].pct_change())/2
market_returns.index = pd.to_datetime(market_returns.index.strftime('%Y-%m-%d'))
market_returns = market_returns.dropna()
# market_returns_mean = market_returns.mean()

# store the average market returns in a DataFrame
market_returns_df = pd.DataFrame(market_returns)

# display(market_returns_df.index)

In [ ]:
common_dates = market_returns_df.index.intersection(ticker_returns.index)

market_returns_df = market_returns_df.loc[common_dates]
ticker_returns = ticker_returns.loc[common_dates]

months = market_returns_df.resample('MS').bfill().index

for month in months:
    temp = market_returns_df[(market_returns_df.index.month == month.month) & (market_returns_df.index.year == month.year)].index
    if len(temp) < 18:
        market_returns_df.drop(index=temp, inplace=True)
        ticker_returns.drop(index=temp, inplace=True)

# print([val for val in ticker_returns.index if val not in market_returns_df.index])

# print(len(market_returns_df))
# print(len(ticker_returns))

In [146]:

ticker_lst = top_24
print(ticker_lst)
index_lst = ["XIU.TO","^GSPC"]

exch_rate = yf.download("CADUSD=x",start="2024-11-20", end="2024-11-21")["Close"].iloc[0] #MUST BE CHANGED TO PULL NOVEMBER 22ND

#Takes a yf.Ticker "ticker", checks if the ticker is in canadian dollars. If so, it will just do a simple API pull for the price history.
#If the price is listed in USD, a conversion operation will simply be applied to each price.
def yfin_pull_convert_USD(ticker):
    listed_currency = ticker.fast_info["currency"]
    if listed_currency == "CAD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
    if listed_currency == "USD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"] * (1/exch_rate)
    else: print("ticker currency is not in USD or CAD error")

#Test cases
#display(yf.Ticker("NVDA").history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
#yfin_pull_convert_USD(yf.Ticker("NVDA"))

#Main function to store the stock prices of the ticker
def gen_tickers(tlist, ilist):
    ret_dataframe = pd.DataFrame()
    for i_str in ilist:
        index_ticker = yf.Ticker(i_str)
        ret_dataframe[i_str] = yfin_pull_convert_USD(index_ticker)
    
    for t_str in tlist:
        ticker = yf.Ticker(t_str)
        ret_dataframe[t_str] = yfin_pull_convert_USD(ticker)
    return ret_dataframe

#Function call: stores a dataframe of index values and stock values
stock_values = gen_tickers(ticker_lst,index_lst)
display(stock_values)

[*********************100%***********************]  1 of 1 completed

['KO', 'RY.TO', 'PG', 'CL', 'T.TO', 'PEP', 'TD.TO', 'BK', 'LMT', 'ABT', 'UNP', 'MO', 'BLK', 'PM', 'MRK', 'AIG', 'AAPL', 'ABBV', 'ACN', 'UNH', 'AXP', 'BAC', 'PFE', 'BIIB']


,XIU.TO,^GSPC,KO,RY.TO,PG,CL,T.TO,PEP,TD.TO,BK,...,MRK,AIG,AAPL,ABBV,ACN,UNH,AXP,BAC,PFE,BIIB
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-02 00:00:00-04:00,28.043184,5984.447960,75.099634,110.245743,196.451105,95.981256,20.027452,228.953331,75.579323,56.477155,...,139.411846,82.054642,240.968586,197.608947,421.622261,707.311640,205.287646,36.248025,44.041071,352.866103
2023-10-03 00:00:00-04:00,27.793226,5902.197278,74.287457,107.573990,196.857004,95.587878,20.149286,226.963846,74.864975,55.470311,...,138.351455,80.127956,239.096301,196.515944,418.845740,700.301263,199.203734,35.175515,43.989170,350.703074
2023-10-04 00:00:00-04:00,27.831684,5950.062652,74.504035,108.572327,197.506524,95.709965,20.795938,228.601454,74.667595,55.362917,...,138.895252,81.111797,240.843763,196.862541,423.167838,701.882060,201.373662,35.216247,43.223579,358.866757
2023-10-05 00:00:00-04:00,27.995117,5942.303590,70.903376,109.409042,194.570366,93.960160,21.198927,216.678081,75.533386,55.967017,...,140.839255,82.191283,242.577342,196.542625,423.537112,709.593381,202.835364,35.379154,43.431202,362.508991
2023-10-06 00:00:00-04:00,28.120096,6012.511272,71.932131,109.247398,194.962722,94.177195,21.133322,216.935202,75.923470,56.584549,...,141.219911,83.175113,246.155482,197.595659,426.997499,721.387161,205.910527,35.392734,42.990009,367.365346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13 00:00:00-05:00,37.814503,8352.596962,87.916493,172.429993,232.462374,127.088174,21.809999,229.894659,78.559998,108.304747,...,137.456739,106.183581,314.154928,237.723415,516.641971,845.491511,401.401609,64.011578,37.287756,231.359917
2024-11-14 00:00:00-05:00,37.883923,8302.065965,87.288517,172.050003,233.160124,127.618461,21.340000,230.466798,79.639999,108.248926,...,137.261370,105.709117,318.480986,236.718653,505.268655,827.740793,402.322601,64.053447,36.310908,230.103976
2024-11-15 00:00:00-05:00,37.655827,8192.449722,86.158165,171.130005,236.593042,130.562966,21.389999,221.354186,78.800003,109.253688,...,134.400591,105.737022,313.987475,230.243534,493.406906,826.456872,400.327046,65.239620,34.608396,223.266035


In [147]:
market_label = "Market Returns"
#Function that transforms a list of values into a list of percent returns.
#Also adds a column of the simple average returns of the index tickers.
def convert_pct_returns(stock_value_df):
    ret_dataframe = stock_value_df.interpolate().pct_change()
    ret_dataframe.dropna(inplace=True)
    ret_dataframe[market_label] = ret_dataframe[index_lst].mean(axis=1)
    df_col_order = index_lst + ["Market Returns"] + ticker_lst
    ret_dataframe = ret_dataframe[df_col_order]
    return ret_dataframe

stock_returns = convert_pct_returns(stock_values)

display(stock_returns)

,XIU.TO,^GSPC,Market Returns,KO,RY.TO,PG,CL,T.TO,PEP,TD.TO,...,MRK,AIG,AAPL,ABBV,ACN,UNH,AXP,BAC,PFE,BIIB
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-03 00:00:00-04:00,-0.008913,-0.013744,-0.011329,-0.010815,-0.024235,0.002066,-0.004098,0.006083,-0.008689,-0.009452,...,-0.007606,-0.023481,-0.007770,-0.005531,-0.006585,-0.009911,-0.029636,-0.029588,-0.001178,-0.006130
2023-10-04 00:00:00-04:00,0.001384,0.008110,0.004747,0.002915,0.009280,0.003299,0.001277,0.032093,0.007215,-0.002636,...,0.003931,0.012278,0.007309,0.001764,0.010319,0.002257,0.010893,0.001158,-0.017404,0.023278
2023-10-05 00:00:00-04:00,0.005872,-0.001304,0.002284,-0.048328,0.007707,-0.014866,-0.018282,0.019378,-0.052158,0.011595,...,0.013996,0.013309,0.007198,-0.001625,0.000873,0.010987,0.007259,0.004626,0.004803,0.010149
2023-10-06 00:00:00-04:00,0.004464,0.011815,0.008140,0.014509,-0.001477,0.002017,0.002310,-0.003095,0.001187,0.005164,...,0.002703,0.011970,0.014751,0.005358,0.008170,0.016620,0.015161,0.000384,-0.010158,0.013397
2023-10-10 00:00:00-04:00,0.014359,0.011545,0.012952,0.016748,0.005309,0.004789,0.000864,0.005322,0.025641,0.009022,...,-0.002599,0.004929,0.005071,0.004385,0.000416,-0.001086,0.010782,0.036057,0.001207,0.003381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13 00:00:00-05:00,0.004214,0.000232,0.002223,-0.003165,-0.001852,0.004462,-0.004699,-0.003199,0.002434,0.002936,...,-0.000812,0.016431,0.003969,-0.004325,0.026649,-0.014317,-0.003015,0.000218,0.020237,-0.014797
2024-11-14 00:00:00-05:00,0.001836,-0.006050,-0.002107,-0.007143,-0.002204,0.003002,0.004173,-0.021550,0.002489,0.013747,...,-0.001421,-0.004468,0.013770,-0.004227,-0.022014,-0.020995,0.002294,0.000654,-0.026198,-0.005429
2024-11-15 00:00:00-05:00,-0.006021,-0.013203,-0.009612,-0.012950,-0.005347,0.014723,0.023073,0.002343,-0.039540,-0.010547,...,-0.020842,0.000264,-0.014109,-0.027354,-0.023476,-0.001551,-0.004960,0.018518,-0.046887,-0.029717


In [148]:
#Creating the correlation matrix: takes a dataframe of values and a string of columns to drop to make the correlation matrix
def correl(data, dropvalue):
    data_marketdrop = data.drop(labels=dropvalue, axis=1)
    ret_corr = data_marketdrop.corr()
    return ret_corr

stock_correlations = correl(stock_returns, [market_label] + index_lst)
#Apply a format to the DataFrame
display(stock_correlations.style.background_gradient(cmap='RdYlGn_r'))


,KO,RY.TO,PG,CL,T.TO,PEP,TD.TO,BK,LMT,ABT,UNP,MO,BLK,PM,MRK,AIG,AAPL,ABBV,ACN,UNH,AXP,BAC,PFE,BIIB
KO,1.000000,0.165334,0.497276,0.540060,0.255596,0.631380,0.037031,0.161013,0.352976,0.303536,0.236888,0.383455,0.170194,0.410920,0.209858,0.054436,0.076600,0.339486,0.160194,0.218004,0.057200,0.077843,0.280591,0.247166
RY.TO,0.165334,1.000000,0.037373,0.061091,0.332487,0.107727,0.340577,0.487742,0.115632,0.149556,0.386168,0.113882,0.510565,0.158691,0.094115,0.443565,0.168736,0.119548,0.178240,0.076739,0.443797,0.515986,0.292035,0.319712
PG,0.497276,0.037373,1.000000,0.666967,0.247714,0.471876,-0.099019,0.007228,0.140013,0.316142,0.018867,0.302148,0.020996,0.334468,0.375553,-0.006098,-0.035853,0.248611,0.048838,0.140372,-0.083319,-0.003339,0.082341,0.177219
CL,0.540060,0.061091,0.666967,1.000000,0.175426,0.435998,-0.049583,0.063166,0.114672,0.302998,0.068292,0.294796,0.115856,0.346038,0.272470,0.060665,-0.040824,0.207075,0.006012,0.129953,-0.022998,-0.020073,0.133494,0.121729
T.TO,0.255596,0.332487,0.247714,0.175426,1.000000,0.190397,0.177393,0.201942,0.153719,0.198152,0.185257,0.232229,0.233648,0.293000,0.158074,0.107023,0.076818,0.118480,0.208333,0.158322,0.056094,0.171975,0.205984,0.299161
PEP,0.631380,0.107727,0.471876,0.435998,0.190397,1.000000,0.026381,0.128030,0.286734,0.213839,0.103110,0.301072,0.079625,0.284972,0.186754,0.037458,0.046944,0.242642,0.113730,0.223449,0.027314,0.073338,0.175694,0.253467
TD.TO,0.037031,0.340577,-0.099019,-0.049583,0.177393,0.026381,1.000000,0.372961,0.107275,0.043875,0.261609,0.144158,0.335131,0.162385,0.082586,0.244648,0.072486,0.063784,0.177147,0.117612,0.390865,0.423686,0.257411,0.207544
BK,0.161013,0.487742,0.007228,0.063166,0.201942,0.128030,0.372961,1.000000,0.105742,0.172085,0.362483,0.147601,0.529446,0.189463,0.102518,0.462027,0.082733,0.102135,0.237996,0.001745,0.508070,0.589852,0.197874,0.255265
LMT,0.352976,0.115632,0.140013,0.114672,0.153719,0.286734,0.107275,0.105742,1.000000,0.234418,0.207871,0.134113,0.028955,0.024816,0.014817,0.156552,-0.061539,0.085526,0.052282,0.191759,0.107174,0.135851,0.129810,0.065530
ABT,0.303536,0.149556,0.316142,0.302998,0.198152,0.213839,0.043875,0.172085,0.234418,1.000000,0.213222,0.127234,0.200165,0.252581,0.194991,0.006239,-0.072344,0.255487,0.150286,0.062528,0.038734,0.155335,0.160307,0.219569


In [149]:


#finding the ticker coordinates of the lowest correlation value (AI)
def low_correl_ticker_pairs(correlation_matrix):
    if correlation_matrix.shape[0] > 1:
        correl_pairs = correlation_matrix.unstack()
        correl_pairs = correl_pairs[correl_pairs.index.get_level_values(0) != correl_pairs.index.get_level_values(1)]
        lowest_corr_pair = correl_pairs.idxmin()
        lowest_corr_value = correl_pairs.min()
        return list(lowest_corr_pair)
    else:
        remaining_column_ticker = correlation_matrix.columns[0]
        return [remaining_column_ticker]

#recursing through the entire correlation matrix to extract every correlation pair
def corr_pair_extract(correl_matrix):
    #creating a list to store low-correlation pairs into
    low_corr_pairs = []
    while correl_matrix.shape[0] > 0:
        pair = low_correl_ticker_pairs(correl_matrix)
        #removing the tickers after they are packaged into a correlation pair
        if isinstance(pair, str):
            ticker1 = pair
            correl_matrix = correl_matrix.drop(ticker1, axis=0)
            correl_matrix = correl_matrix.drop(ticker1, axis=1)
        else: 
            low_corr_pairs.append(pair)
            correl_matrix = correl_matrix.drop(pair, axis=0)
            correl_matrix = correl_matrix.drop(pair, axis=1)
    return low_corr_pairs

#running the correlation pairing function and storing it in variable "stock pairs"
stock_pairs = corr_pair_extract(stock_correlations)
print(stock_pairs)

[['PG', 'TD.TO'], ['ABT', 'AAPL'], ['CL', 'AXP'], ['MRK', 'UNH'], ['LMT', 'PM'], ['PEP', 'AIG'], ['ABBV', 'ACN'], ['KO', 'BAC'], ['RY.TO', 'MO'], ['UNP', 'PFE'], ['T.TO', 'BK'], ['BLK', 'BIIB']]


In [172]:
#Arbitrarily assign a seed to keep results consistent
random.seed(21152764)

#This part creates lists of stocks, ranging from sizes 12 to 24, 5 of each size, which randomly selects stocks.
def make_port_list(stock_pairs):
    ret_list = []
    #iterates through the minimum selectable stock pairs (6) and the maximum (length of the stock pairs)
    print(len(stock_pairs))
    for i in range(6, len(stock_pairs)):
        print(i)
        #simple iteration of 5 times
        for j in range(0,5):
            #generates a random list of indexes to pick from the stock pairs
            rand_list = random.sample(range(0,len(stock_pairs)-1), i)
            portfolio = []
            for k in rand_list:
                portfolio += stock_pairs[k]
            ret_list.append(portfolio)
    return ret_list

port_list = make_port_list(stock_pairs)
#display(port_list)
#print(list(map(len, port_list)))

#len(port_list)

12
6
7
8
9
10
11


In [151]:
stock_values

,XIU.TO,^GSPC,KO,RY.TO,PG,CL,T.TO,PEP,TD.TO,BK,...,MRK,AIG,AAPL,ABBV,ACN,UNH,AXP,BAC,PFE,BIIB
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-02 00:00:00-04:00,28.043184,5984.447960,75.099634,110.245743,196.451105,95.981256,20.027452,228.953331,75.579323,56.477155,...,139.411846,82.054642,240.968586,197.608947,421.622261,707.311640,205.287646,36.248025,44.041071,352.866103
2023-10-03 00:00:00-04:00,27.793226,5902.197278,74.287457,107.573990,196.857004,95.587878,20.149286,226.963846,74.864975,55.470311,...,138.351455,80.127956,239.096301,196.515944,418.845740,700.301263,199.203734,35.175515,43.989170,350.703074
2023-10-04 00:00:00-04:00,27.831684,5950.062652,74.504035,108.572327,197.506524,95.709965,20.795938,228.601454,74.667595,55.362917,...,138.895252,81.111797,240.843763,196.862541,423.167838,701.882060,201.373662,35.216247,43.223579,358.866757
2023-10-05 00:00:00-04:00,27.995117,5942.303590,70.903376,109.409042,194.570366,93.960160,21.198927,216.678081,75.533386,55.967017,...,140.839255,82.191283,242.577342,196.542625,423.537112,709.593381,202.835364,35.379154,43.431202,362.508991
2023-10-06 00:00:00-04:00,28.120096,6012.511272,71.932131,109.247398,194.962722,94.177195,21.133322,216.935202,75.923470,56.584549,...,141.219911,83.175113,246.155482,197.595659,426.997499,721.387161,205.910527,35.392734,42.990009,367.365346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13 00:00:00-05:00,37.814503,8352.596962,87.916493,172.429993,232.462374,127.088174,21.809999,229.894659,78.559998,108.304747,...,137.456739,106.183581,314.154928,237.723415,516.641971,845.491511,401.401609,64.011578,37.287756,231.359917
2024-11-14 00:00:00-05:00,37.883923,8302.065965,87.288517,172.050003,233.160124,127.618461,21.340000,230.466798,79.639999,108.248926,...,137.261370,105.709117,318.480986,236.718653,505.268655,827.740793,402.322601,64.053447,36.310908,230.103976
2024-11-15 00:00:00-05:00,37.655827,8192.449722,86.158165,171.130005,236.593042,130.562966,21.389999,221.354186,78.800003,109.253688,...,134.400591,105.737022,313.987475,230.243534,493.406906,826.456872,400.327046,65.239620,34.608396,223.266035


In [152]:
stock_values_just_date = stock_values
stock_values_just_date.index = pd.to_datetime(stock_values_just_date.index)
stock_values_just_date.index = stock_values_just_date.index.date

# Display the updated DataFrame
display(stock_values_just_date)


,XIU.TO,^GSPC,KO,RY.TO,PG,CL,T.TO,PEP,TD.TO,BK,...,MRK,AIG,AAPL,ABBV,ACN,UNH,AXP,BAC,PFE,BIIB
2023-10-02,28.043184,5984.447960,75.099634,110.245743,196.451105,95.981256,20.027452,228.953331,75.579323,56.477155,...,139.411846,82.054642,240.968586,197.608947,421.622261,707.311640,205.287646,36.248025,44.041071,352.866103
2023-10-03,27.793226,5902.197278,74.287457,107.573990,196.857004,95.587878,20.149286,226.963846,74.864975,55.470311,...,138.351455,80.127956,239.096301,196.515944,418.845740,700.301263,199.203734,35.175515,43.989170,350.703074
2023-10-04,27.831684,5950.062652,74.504035,108.572327,197.506524,95.709965,20.795938,228.601454,74.667595,55.362917,...,138.895252,81.111797,240.843763,196.862541,423.167838,701.882060,201.373662,35.216247,43.223579,358.866757
2023-10-05,27.995117,5942.303590,70.903376,109.409042,194.570366,93.960160,21.198927,216.678081,75.533386,55.967017,...,140.839255,82.191283,242.577342,196.542625,423.537112,709.593381,202.835364,35.379154,43.431202,362.508991
2023-10-06,28.120096,6012.511272,71.932131,109.247398,194.962722,94.177195,21.133322,216.935202,75.923470,56.584549,...,141.219911,83.175113,246.155482,197.595659,426.997499,721.387161,205.910527,35.392734,42.990009,367.365346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13,37.814503,8352.596962,87.916493,172.429993,232.462374,127.088174,21.809999,229.894659,78.559998,108.304747,...,137.456739,106.183581,314.154928,237.723415,516.641971,845.491511,401.401609,64.011578,37.287756,231.359917
2024-11-14,37.883923,8302.065965,87.288517,172.050003,233.160124,127.618461,21.340000,230.466798,79.639999,108.248926,...,137.261370,105.709117,318.480986,236.718653,505.268655,827.740793,402.322601,64.053447,36.310908,230.103976
2024-11-15,37.655827,8192.449722,86.158165,171.130005,236.593042,130.562966,21.389999,221.354186,78.800003,109.253688,...,134.400591,105.737022,313.987475,230.243534,493.406906,826.456872,400.327046,65.239620,34.608396,223.266035
2024-11-18,37.725246,8224.546219,86.325624,171.600006,238.281606,132.042203,21.629999,220.949500,78.870003,109.756068,...,134.749466,106.490593,318.201891,232.043720,492.318418,822.856544,398.471050,65.197757,34.692128,220.405256


In [162]:
port_list[-1]

['PEP',
 'AIG',
 'LMT',
 'PM',
 'RY.TO',
 'MO',
 'UNP',
 'PFE',
 'T.TO',
 'BK',
 'PG',
 'TD.TO',
 'ABBV',
 'ACN',
 'MRK',
 'UNH',
 'KO',
 'BAC',
 'CL',
 'AXP',
 'ABT',
 'AAPL']

In [161]:
portfolio_data = []
def portfolio_generator(list_of_port):
    global portfolio_data
   
    # this dictionary will store Ticker closing prices on the target dat

    # creating portfolios -- starting wtih 12 stocks, going upto 24 stocks
    for i in range(len(port_list)):

        # for the ith portfolio, select the first i tickers
        selected_tickers = port_list[i]
        
        # objective function -- will be used in optimization steps
        # returns the tracking error between the portfolio's returns and the market's returns
        def objective(weights):
            return np.std(np.dot(ticker_returns[selected_tickers].dropna().values, weights) - market_returns_df.dropna().values)
        
        # initial weights are calcualted so that, to start, every stock has equal weight
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        # this constraint ensures that all of the stock weights sum to 1 (100%)
        def sum_weights(weights):
            return sum(weights) - 1
        
        constraints = [{'type': 'eq', 'fun': sum_weights}]
        
        # the bounds for each stock are that each stock must have a minimum weight of 1/2n% and maximum weight of 15%
        bounds = [(1 / (2 * len(selected_tickers)), 0.15)] * len(selected_tickers)
        
        # optimize the portfolio weights
        result = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)
        
        if result.success:

            weights = result.x
                        
            # calculate portfolio value and the number of shares to purchase
            portfolio_value = 1000000 * weights
            shares = portfolio_value / np.array([stock_values_just_date.at[pd.to_datetime(TARGET_DATE).date(), ticker] for ticker in selected_tickers]) 

            # calculate fees
            fee = np.minimum(3.95, 0.001 * shares)
            
            # subtract fees from portfolio value
            portfolio_value -= fee
            
            # Add to the results
            portfolio_data.append({
                'tickers': ', '.join(selected_tickers),  # Tickers in the portfolio
                'weights': result.x,  # Optimized weights
                'Tracking Error': result.fun,  # Standard deviation (risk)
                'num_stocks': i,  # Number of stocks in the portfolio
                'total_cost': portfolio_value.sum() + sum(fee),  # Total cost after fees
                'shares_purchased': shares,  # Shares purchased
                'portfolio_value': portfolio_value.sum(),  # Total portfolio value after fees
                'final_fees': sum(fee)
            })

    return pd.DataFrame(portfolio_data)

portfolio_df = portfolio_generator(port_list)
portfolio_df

,tickers,weights,Tracking Error,num_stocks,total_cost,shares_purchased,portfolio_value,final_fees
0,"CL, AXP, PEP, AIG, LMT, PM, KO, BAC, RY.TO, MO...","[0.10556876629738281, 0.04166666666666667, 0.0...",0.008735,0,1.000000e+06,"[830.6734024998711, 103.80293896234475, 334.78...",999993.609031,6.390969
1,"MRK, UNH, ABT, AAPL, PEP, AIG, UNP, PFE, RY.TO...","[0.11722410278210449, 0.06952718620606113, 0.0...",0.008633,1,1.000000e+06,"[852.8072438049373, 82.23286137002081, 607.475...",999993.726973,6.273027
2,"ABT, AAPL, ABBV, ACN, RY.TO, MO, CL, AXP, KO, ...","[0.06777670795373787, 0.13356985211071856, 0.0...",0.008556,2,1.000000e+06,"[419.6306427306155, 425.1719140015237, 246.317...",999994.082419,5.917581
3,"T.TO, BK, ABBV, ACN, UNP, PFE, CL, AXP, ABT, A...","[0.12017545604921244, 0.05611302317770668, 0.0...",0.008698,3,1.000000e+06,"[5510.108161023736, 518.1030805291181, 331.468...",999989.959260,10.040740
4,"ABBV, ACN, RY.TO, MO, LMT, PM, UNP, PFE, MRK, ...","[0.05688261521890825, 0.07835470364226804, 0.1...",0.008754,4,1.000000e+06,"[239.28065863564197, 151.6615143603405, 869.91...",999993.311380,6.688620
5,"RY.TO, MO, UNP, PFE, PEP, AIG, T.TO, BK, ABT, ...","[0.07539183659800762, 0.05789000981333496, 0.0...",0.008687,5,1.000000e+06,"[437.2315710745653, 750.8298577005979, 106.453...",999989.051166,10.948834
6,"KO, BAC, RY.TO, MO, T.TO, BK, PG, TD.TO, MRK, ...","[0.12154677162074774, 0.035714285714285726, 0....",0.008689,6,1.000000e+06,"[1382.5252517160245, 557.9347767717735, 705.81...",999988.454832,11.545152
7,"LMT, PM, RY.TO, MO, KO, BAC, PG, TD.TO, ABT, A...","[0.1489650837341594, 0.0357952231155486, 0.090...",0.008439,7,1.000000e+06,"[191.39504421663912, 204.81049913033092, 526.6...",999993.616741,6.383259
8,"ABT, AAPL, MRK, UNH, ABBV, ACN, RY.TO, MO, UNP...","[0.06401044454204921, 0.15, 0.0995451745418138...",0.008435,8,1.000000e+06,"[396.3123142981034, 477.4714210760944, 724.192...",999994.581736,5.418264
9,"UNP, PFE, T.TO, BK, PEP, AIG, CL, AXP, ABT, AA...","[0.03571428571428571, 0.03571428571428571, 0.0...",0.008420,9,1.000000e+06,"[106.45342217353074, 957.8019562968941, 4449.0...",999991.032027,8.967973


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

ALEX
- coded get_sector function
- edited validity function
- matched indexes of DataFrames (the first containing the market returns, and the second containing ticker returns)

DEREK
- coded correlation function

SHARUGA
- coded validity function
- coded portfolio_generator function